In [18]:
from time import time
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import cv2
import numpy as np
from PIL import Image
from keras import optimizers
import keras.layers.advanced_activations
import scipy
import random
import matplotlib.pyplot as plt  
from scipy.stats import norm  
import keras
from keras.models import Sequential
from keras.initializers import VarianceScaling,RandomNormal
from keras.layers import Input, Dense, Dropout, Conv2D, MaxPooling2D, concatenate
from keras.layers import UpSampling2D, Reshape, Lambda, Flatten, Activation,Concatenate,Add
from keras.models import Model  
from keras.optimizers import SGD, Adadelta, Adagrad,Adam
from keras import backend as K  
from keras import objectives  
from keras.utils.vis_utils import plot_model  
from keras.utils import np_utils, generic_utils
import sys 
import tensorflow as tf
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.layers import BatchNormalization
from keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sklearn
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter
%matplotlib inline
from sklearn import manifold, datasets,cluster
from sklearn.utils import check_random_state
from sklearn import metrics

In [41]:
train_img = np.load("img_set_log.npy").reshape([-1,256,256,1])
train_seg = np.load("seg_set_log.npy").reshape([-1,256,256,1])
train_flo = np.load("flo_set_rgb_log.npy")
train_rflo = np.load("rflo_set_rgb_log.npy")
#train_img = np.log(train_img+1.0)
train_img = train_img/train_img.max()
#train_seg = np_utils.to_categorical(train_seg, 2)

In [47]:
def translate(image, x, y):
    # 定义平移矩阵
    M = np.float32([[1, 0, x], [0, 1, y]])
    shifted = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))

    # 返回转换后的图像
    return shifted

In [48]:
test_img = train_img[1150:]
test_seg = train_seg[1150:]
test_flo = train_flo[1150:]
test_rflo = train_rflo[1150:]
train_img = train_img[:1150]
train_seg = train_seg[:1150]
train_flo = train_flo[:1150]
train_rflo = train_rflo[:1150]

# 对train进行flip, shift变换

In [49]:
train_img_flip = []
train_seg_flip = []
train_flo_flip = []
train_rflo_flip = []
for i in range(1150):
    train_img_flip.append(cv2.flip(train_img[i],1))
    train_seg_flip.append(cv2.flip(train_seg[i],1))
    train_flo_flip.append(cv2.flip(train_flo[i],1))
    train_rflo_flip.append(cv2.flip(train_rflo[i],1))

In [50]:
np.shape(train_img_flip)

(1150, 256, 256)

In [57]:
train_img = np.vstack([train_img[:,:,:,0],train_img_flip])
train_seg = np.vstack([train_seg[:,:,:,0],train_seg_flip])
train_flo = np.vstack([train_flo,train_flo_flip])
train_rflo = np.vstack([train_rflo,train_rflo_flip])

In [59]:
np.shape(train_img)

(2300, 256, 256)

In [60]:
np.shape(train_seg)

(2300, 256, 256)

In [74]:
train_img_shift = []
train_seg_shift = []
train_flo_shift = []
train_rflo_shift = []
for i in range(2300):
    #向上2
    train_img_shift.append(translate(train_img[i],0,-2))
    train_seg_shift.append(np.round(translate(train_seg[i]+0.00001,0,-2)))
    train_flo_shift.append(translate(train_flo[i],0,-2))
    train_rflo_shift.append(translate(train_rflo[i],0,-2))
    
    train_img_shift.append(translate(train_img[i],2,0))
    train_seg_shift.append(np.round(translate(train_seg[i]+0.00001,2,0)))
    train_flo_shift.append(translate(train_flo[i],2,0))
    train_rflo_shift.append(translate(train_rflo[i],2,0))
    
    train_img_shift.append(translate(train_img[i],0,2))
    train_seg_shift.append(np.round(translate(train_seg[i]+0.00001,0,2)))
    train_flo_shift.append(translate(train_flo[i],0,2))
    train_rflo_shift.append(translate(train_rflo[i],0,2))  
    
    train_img_shift.append(translate(train_img[i],1,1))
    train_seg_shift.append(np.round(translate(train_seg[i]+0.00001,1,1)))
    train_flo_shift.append(translate(train_flo[i],1,1))
    train_rflo_shift.append(translate(train_rflo[i],1,1))    
    
    train_img_shift.append(translate(train_img[i],1,-1))
    train_seg_shift.append(np.round(translate(train_seg[i]+0.00001,1,-1)))
    train_flo_shift.append(translate(train_flo[i],1,-1))
    train_rflo_shift.append(translate(train_rflo[i],1,-1))  

In [75]:
np.shape(train_img_shift)

(11500, 256, 256)

In [76]:
train_img = np.vstack([train_img,train_img_shift])
train_seg = np.vstack([train_seg,train_seg_shift])
train_flo = np.vstack([train_flo,train_flo_shift])
train_rflo = np.vstack([train_rflo,train_rflo_shift])

In [77]:
np.shape(train_img)

(13800, 256, 256)

In [78]:
np.save("train_img_aug.npy",train_img)
np.save("train_seg_aug.npy",train_seg)
np.save("train_flo_aug.npy",train_flo)
np.save("train_rflo_aug.npy",train_rflo)
np.save("test_img.npy",test_img)
np.save("test_seg.npy",test_seg)
np.save("test_flo.npy",test_flo)
np.save("test_rflo.npy",test_rflo)

In [80]:
np.shape(test_flo)

(127, 256, 256, 3)